In [1]:
!nvidia-smi 

Tue Feb 28 23:35:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone 

fatal: You must specify a repository to clone.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --verbose         be more verbose
    -q, --quiet           be more quiet
    --progress            force progress reporting
    --reject-shallow      don't clone shallow repository
    -n, --no-checkout     don't create a checkout
    --bare                create a bare repository
    --mirror              create a mirror repository (implies bare)
    -l, --local           to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    -s, --shared          setup as shared repository
    --recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    --recursive ...       alias of --recurse-submodules
    -j, --jobs <n>        number of submodules cloned in parallel
    --template <template-directory>
                          directory from which templates will be used
    --reference <repo>    reference rep

In [3]:
!lscpu 

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz
Stepping:                        3
CPU MHz:                         2000.174
BogoMIPS:                        4000.34
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        1 MiB
L3 cache:                        38.5 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 

In [4]:
!tar -xf /kaggle/input/idwint/aidw.tar.xz

In [5]:
data_path_reg = "/kaggle/input/idwint/ValidationFev2023/ValidationFev2023/*.dtable"

In [6]:
from scipy.spatial import distance_matrix
import jax
import numpy as np

from aidw.utils import Grid
from aidw.dtable import DTable
import aidw.interpolator as interp

        
def interpolate(
        self,
        data: "DTable",
        search_radius: float,
        alpha: float,
        inplace=False,
        num_rows: int = 10,
        num_cols: int = 10,
        return_rms: bool = True,
        resolution: float = 1.0,
        return_column: str = "ipt_value",
        save_path: str = None,
    ):
        avg_lat = None
        if not self.metric:
            self.convert_geometric_to_metric(self.avg_lat, inplace)
            data.convert_geometric_to_metric(self.avg_lat, inplace)
        points = data.data[[data.latitude, data.longitude]].values

        xy = self.data[[self.latitude, self.longitude]].values
        values = self.data[self.value].values

        if self.acp:
            if not inplace:
                self.latitude = self.latitude + " (acp)"
                self.longitude = self.longitude + " (acp)"

            grid, ew, ev = Grid.from_points(
                xy, values, num_rows, num_cols, resolution, self.acp
            )

            points = points @ ev
        else:
            grid = Grid.from_points(xy, values, num_rows, num_cols, resolution, False)

        vals = interp.compute_inverse_distance_weighing_value_with_grid(
            points, grid, search_radius, alpha
        )
        data.data[return_column] = jax.device_get(vals)
        data._update_type_list()

        if save_path:
            data.save(save_path)
DTable.interpolate = interpolate


def find_optimal_alpha_radius(
            self,
            ratio= 0.2,
            alpha_min: float = 1.0,
            alpha_max: float = 2.0,
            alpha_step: float = 0.1,
            radius_min: float = None,
            radius_max: float = None,
            radius_step: float = 50,
            save_path= None,
            num_rows: int = 20,
            num_cols: int = 20,
            resolution: float = 1.0,
            acp: bool = True,
            low_memory: bool = False,
            save_sample = None,
        ):

            if not self.metric:
                self.convert_geometric_to_metric(self.data[self.latitude].mean())

            matrix = self.data[[self.latitude, self.longitude]].values
            values = self.data[self.value].values

            assert ratio < 1, "Le ratio d'échantillonage doit être inférieur à 1"
            num_point = int(ratio * (len(matrix)))

            indices = np.arange(len(matrix))
            np.random.shuffle(indices)

            distance = distance_matrix(matrix, matrix)
            distance[distance == 0] = np.nan
            dmin = np.nanmin(distance) if radius_min == None else radius_min
            dmax = np.nanmax(distance) if radius_max == None else radius_max
            ipt = matrix[indices[:num_point]]
            true = values[indices[:num_point]]

            matrix = matrix[indices[num_point:]]
            values = values[indices[num_point:]]
            radius = np.arange(dmin, dmax, radius_step)
            alpha = np.arange(alpha_min, alpha_max, alpha_step)

            num_radius = radius.shape[0]
            num_alpha = alpha.shape[0]
            radius = jax.device_put(radius)
            alpha = jax.device_put(alpha)

            if acp:
                grid, _, ev = Grid.from_points(
                    matrix, values, num_rows, num_cols, resolution, True
                )
                ipt = ipt @ ev
            else:
                grid = Grid.from_points(
                    matrix, values, num_rows, num_cols, resolution, False
                )

            alpha_, radius_, rms, all_rms = interp.find_optimal_alpha_radius(
                ipt,
                true,
                grid,
                alpha,
                radius,
                return_rms=True,
                low_memory=False,  # low_memory,
            )

            alpha = np.repeat(alpha, num_radius)
            radius = np.tile(radius, num_alpha)

            all_rms = all_rms.reshape(-1)[::-1]

            if save_sample:
                data = DTable(
                    self.data.iloc[indices[:num_point]],
                    self.latitude,
                    self.longitude,
                    self.value,
                    self.delimiter,
                    self.decimal,
                    metric=self.metric,
                    type_list=self.type_list,
                )

                non_data = DTable(
                    self.data.iloc[indices[num_point:]],
                    self.latitude,
                    self.longitude,
                    self.value,
                    self.delimiter,
                    self.decimal,
                    metric=self.metric,
                    type_list=self.type_list,
                )
                # non_data.save(f"{save_sample}-pts")
                
                non_data.interpolate(
                    data, radius_, alpha_, num_cols=num_cols, num_rows=num_rows,
                    #save_path=save_sample
                )
                

                data.save(save_sample)

            if save_path:
                df = pd.DataFrame(
                    {
                        "alpha": alpha,
                        "radius": radius,
                        "rms": all_rms,
                    }
                )
                df.to_csv(save_path, index=False)

            return alpha_, radius_, rms
DTable.find_optimal_alpha_radius = find_optimal_alpha_radius

In [7]:
from glob import glob
from time import time
from aidw.dtable import DTable
import pandas as pd
from tqdm.notebook import tqdm
import os

if os.path.exists("files.txt"):
    with open("files.txt", "r") as f:
        files = f.readlines()
else:
    files = []

In [8]:
!mkdir -p output/{10,20,30,40,50}/{dtable,stats,diagram}
!tree -L 4 output

output
├── 10
│   ├── diagram
│   ├── dtable
│   └── stats
├── 20
│   ├── diagram
│   ├── dtable
│   └── stats
├── 30
│   ├── diagram
│   ├── dtable
│   └── stats
├── 40
│   ├── diagram
│   ├── dtable
│   └── stats
└── 50
    ├── diagram
    ├── dtable
    └── stats

20 directories, 0 files


In [9]:
with open("/kaggle/input/idwint/files.txt") as f:
    files = f.readlines()
    
files = [file.strip() for file in files]
files.sort()
files

['/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts100_repN2.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts1600_repN2.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts1600_repN3.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts200_repN1.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts200_repN2.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts200_repN3.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts3200_repN2.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts3200_repN3.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts400_repN2.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts400_repN3.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts50_

In [10]:
datafiles = glob('/kaggle/input/validation/ValidationFev2023_FaibleEch/*.dtable')
datafiles.sort()
datafiles

['/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts100_repN1.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts100_repN2.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts100_repN3.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts1600_repN1.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts1600_repN2.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts1600_repN3.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts200_repN1.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts200_repN2.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts200_repN3.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts3200_repN1.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts320

In [11]:
files_  = []
for f in datafiles:
    if f not in files:
        files_.append(f)

In [12]:
files = files_

In [13]:
files

['/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts100_repN1.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts100_repN3.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts1600_repN1.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts3200_repN1.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts400_repN1.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts50_repN1.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts800_repN1.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts800_repN2.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit100_Npts100_repN2.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit100_Npts1600_repN2.dtable',
 '/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit100_Npt

In [14]:
len(files)

88

In [15]:
from glob import glob
from time import time
from aidw.dtable import DTable
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
plt.close()

start = time()
for file in tqdm(files[:len(files)//2]):
        file = file.strip().split('/')[-1]
        
        file = f"/kaggle/input/validation/ValidationFev2023_FaibleEch/{file}"
        end = time()
        spend = (end - start)/3600

        if spend>10:
            break
                
        data = DTable(
            file,
            0,
            1,
            3,
            ";",
            ".",
            acp=False,
            metric=True,
        )
        basename = file.split('/')[-1].split('.')[0]
        for ratio in [10, 20, 30, 40, 50]:
            
            stats = dict(alpha=[], radius=[], rms=[], time=[])
            
            !mkdir  output/$ratio/dtable/$basename
            
            for i in range(101):
                
                start_ = time()
                alpha, radius, rms = data.find_optimal_alpha_radius(
                    ratio / 100, 1, 4.05, 0.05, 10, 500, 10, acp=False,
                    save_sample=f"output/{ratio}/dtable/{basename}/{i}.dtable",
                    save_path=f"output/{ratio}/dtable/{basename}/{i}.csv"
                )
            
                end_ = time()

                stats["alpha"].append(float(alpha))
                stats["radius"].append(float(radius))
                stats["rms"].append(float(rms))
                stats["time"].append(end - start)
                start_ = end_

            
            stats = pd.DataFrame(stats)
            stats.to_csv(f"output/{ratio}/stats/{basename}.csv", index=False)
            df = stats
            plt.hist2d(df.alpha, df.radius, bins=(20, 20), cmap=plt.cm.Reds)
            plt.colorbar()
            plt.title(basename)
            plt.xlabel('alpha')
            plt.ylabel('radius')
            idx = df.idxmin()['rms']

            idx = idx if not np.isnan(idx) else 0
            best = df.iloc[idx]
            plt.scatter([best.alpha], [best.radius], marker="*", label="best")
            plt.legend()
            plt.savefig(f"output/{ratio}/diagram/{basename}.svg")
            plt.plot()
            plt.close()
            
        with open("files.txt", "a") as f:
            print(file, file=f)

  0%|          | 0/44 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [16]:
!ls /

NGC-DL-CONTAINER-LICENSE		mnt
bin					opt
boot					proc
dev					root
entrypoint.sh				run
etc					run_jupyter.sh
home					sbin
kaggle					src
lib					srv
lib32					sys
lib64					tmp
libnvinfer8_8.0.0-1+cuda11.0_amd64.deb	usr
libx32					var
media


In [17]:
%%file README.md

# output

## output/dtable/{dtable}


### files.txt 
Contient la liste des fichiers utilisés 

Writing README.md


In [18]:
!cat files.txt

/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts100_repN1.dtable
/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts100_repN3.dtable
/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts1600_repN1.dtable
/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts3200_repN1.dtable
/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts400_repN1.dtable
/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts50_repN1.dtable
/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts800_repN1.dtable
/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit0_Npts800_repN2.dtable
/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit100_Npts100_repN2.dtable
/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit100_Npts1600_repN2.dtable
/kaggle/input/validation/ValidationFev2023_FaibleEch/Carte5_Bruit100_Npts200_repN2.dtable
/kaggle/input/validation

In [19]:
!tar -cJf all.tar.xz output README.md files.txt

In [20]:
!ls output/50/stats

Carte5_Bruit0_Npts100_repN1.csv     Carte5_Bruit10_Npts50_repN1.csv
Carte5_Bruit0_Npts100_repN3.csv     Carte5_Bruit10_Npts50_repN3.csv
Carte5_Bruit0_Npts1600_repN1.csv    Carte5_Bruit10_Npts800_repN3.csv
Carte5_Bruit0_Npts3200_repN1.csv    Carte5_Bruit200_Npts1600_repN2.csv
Carte5_Bruit0_Npts400_repN1.csv     Carte5_Bruit200_Npts200_repN2.csv
Carte5_Bruit0_Npts50_repN1.csv	    Carte5_Bruit200_Npts400_repN1.csv
Carte5_Bruit0_Npts800_repN1.csv     Carte5_Bruit200_Npts400_repN2.csv
Carte5_Bruit0_Npts800_repN2.csv     Carte5_Bruit200_Npts400_repN3.csv
Carte5_Bruit100_Npts100_repN2.csv   Carte5_Bruit200_Npts50_repN2.csv
Carte5_Bruit100_Npts1600_repN2.csv  Carte5_Bruit200_Npts800_repN1.csv
Carte5_Bruit100_Npts200_repN2.csv   Carte5_Bruit200_Npts800_repN3.csv
Carte5_Bruit100_Npts3200_repN1.csv  Carte5_Bruit50_Npts100_repN1.csv
Carte5_Bruit100_Npts3200_repN2.csv  Carte5_Bruit50_Npts100_repN2.csv
Carte5_Bruit100_Npts400_repN1.csv   Carte5_Bruit50_Npts100_repN3.csv
Carte5_Bruit100_Npts400_repN2